<a href="https://colab.research.google.com/github/Waleed-Daud/IndabaX-Rwanda-RL/blob/master/RL-tutorial.ipynb" 
target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**
In this practical we introduce the idea of reinforcement learning, discuss how it differs from supervised and unsupervised learning and then build an agent that learns to drive a car up the mountain.

# **Learning Objectives**
* Understand what is reinforcement learning and how it differs from supervised and unsupervised learning.
* Understand the relationship between the **environment** and the **agent**.
* Understand what is a **policy** and how a **policy** is used by an agent to select an action.
* Understand how the **state**, **action** and **reward** are communicated between the agent and the environment. 
* Understand **Q-learning** algorithm.
* Understand and Implement **DQN** algorithm.
* Discover at least one potential issue with the DQN.
* See the big picture of Reinforcment Learning.

# **Learning Paradims**
 
#### **1- Supervised learning:**
Is a learning paradaim, given an input and a target value or class. **The goal is to predict the target value.**



<p align="center">
  <img width="850" height="800" src="data/supervised_learning.gif">
</p>

<p align="center">
  <a href="https://becominghuman.ai/building-an-image-classifier-using-deep-learning-in-python-totally-from-a-beginners-perspective-be8dbaf22dd8" > Image by: Venkatesh Tata </a>
</p>
  
#### **2- Unsupervised learning:**
Is a learning paradaim, where **we are only given an input and we don't have the output**. **The goal is to look for patterns in that input.**


<p align="center">
  <img width="800" height="800" src="data/unsupervised.png">
</p>

<p align="center">
  <a href="https://becominghuman.ai/building-an-image-classifier-using-deep-learning-in-python-totally-from-a-beginners-perspective-be8dbaf22dd8" > Image by: Explorez vos données avec des algorithmes non supervisés </a>
</p>

#### **3- Reinforcement learning:** 
Is a learning paradaim, which cares about training an **agent** to maximise a **reward** it obtains through interaction with an **environment**. 


![RL Overview](data/rl-image.png) 

<div> <a > Image by: Benjamin Rosman </a> </div>



#### **How it works?**

The **environment** defines a set of **actions** that an agent can take. The agent observes the current **state** of the environment, tries actions and *learns* a **policy** which is a distribution over the possible actions given a state of the environment. 

<hr>
# **Table of contents**

We will use an OpenAI Gym environment called **Mountain Car**, the goal is to train an agent to drive a car up the mountain. The practical will be as follows:

1. Introduce the **Mountain Car** environment and explore the states and actions available.
2. Create a simple agent that takes random actions.
3. Going from random agent to skilled agent:        
4. Merge Q-learning with Deep Learning:
5. Dive deeper into DQN:    
6. The big picture of Reinforcement learing:
7. References.

<hr>

In [ ]:
! git clone https://github.com/Waleed-Daud/IndabaX-Rwanda-RL.git
% cd IndabaX-Rwanda-RL

In [ ]:
import math
import random

import gym


import numpy as np
import matplotlib.pyplot as plt
from DQN import Agent
from DQN import run
from DQN import Metric


from keras.models import Sequential
from keras import layers
from keras  import optimizers
from keras import losses

# **1- Explore the Environment**

![SegmentLocal](data/environment.gif "segment")

 The goal is to drive up the mountain on the right; however, the car's engine is not strong enough to scale the mountain in a single pass. Therefore, the only way to succeed is to drive back and forth to build up momentum.

Now let us take a look to how the states and actions represented:

**States:**


|Representation |  State |   Min value|  Max value  |
|---|---------------|------|-------|
| 0 |  position |  -1.2 |     0.6 | 
| 1 |   velocity|   -0.07|     0.07 | 


**Actions:**


|Representation |  Action| 
|---|---------------|
| 0 |  push left |
| 1 |   no push|  
| 2 |    push right|  



In [ ]:
mountain_car = gym.make('MountainCar-v0')
# mountain_car = gym.make('CartPole-v0')

num_states = mountain_car.observation_space.shape[0]
num_actions = mountain_car.action_space.n

print("States:  {} Type: Contiuous  Represented as: Vector ".format(num_states,) )
print("Actions: {} Type: Discrete   Represented as: scalar or number ".format(num_actions+1))  # adding one, because action are starting from 0.
print("Example of a state: ", mountain_car.observation_space.sample())
print("Example of an action: ", mountain_car.action_space.sample())

<hr>
# **2- Build a random agent**

**reset:** get an initial state in the envirnoment.

**render:** show the mountain car environemnt (simulator) in the screen.

**step:** apply the action in the environemnt.

**smaple:** sample an action

**close:**  close the envrinment ( simulator ).


**( uncomment and run this in your computer, not in colab)**

In [ ]:
mountain_car.reset()
for _ in range(1000):
    mountain_car.render()
    random_agent = mountain_car.action_space.sample()
    mountain_car.step(random_agent) # take a random action
mountain_car.close()

<hr>
# 3- Going from random agent to skilled agent

### **3.1 Q-learning Theory**

As we mentioned before, the goal of Reinforcement Learning is to train an agent to maximise a reward it obtains through interaction with an environment.

#### **So, what is reward that the agent want to maximize?**

Basically, it is the objective function of the agent, and has another name called Reward function. It formalized matmatically as:

\begin{align}
&& \large  G_t = r_t + \gamma r_{t+1} + \gamma^{2} r_{t+2} + \gamma^{3} r_{t+3} + .... =  \sum_{t=1}^{T_i} \gamma^t r_{i,t}
\end{align}

#### **Q-learning**:

- Is an **Off-Policy** RL algorithm for **Temporal Difference learning**.
- Off-policy: the agent updates the actions values based on a policy that differs from the one that used to interact with the environment.
- On-policy: the agent updates the actions values based on the same policy that use it to interact with the environment.
- Temporal Difference learning (TD) : a reinforcement learning algorithm that similar to the Q-learning but it is on-policy algorithm.

#### **Q-learning considers that every action in a given state has a value $ Q(s_t,a_t)$, this value formalized matmatically as:**

\begin{align}
&& \large Q_n(s_t,a_t) = Q_{n-1}(s_t,a_t) + \alpha [r_{t} + \max_{a} \gamma Q_n(s_{t+1},a) - Q_{n-1}(s_t,a_t) ] 
\end{align}

### **3.2  The intuition behind Q-learning fromula**

**The formula actually saying:** the value of the current action depends on its previous value and a constant multilplied by a linear combination of the discounted maximum action value in the next state **and** the previous action value. 

**What do these terms mean?**

- $ Q_n(s_t,a_t) \equiv \space The \space value \space of \space action \space a_t \space   \space in  \space the  \space current \space  time \space  step \space s_t. $

- $ \max_{a} \gamma Q_n(s_{t+1},a) \equiv \space The \space max \space value \space over \space all \space the \space actions \space in \space the \space next \space state \space s_{t+1}.$

- $\alpha \equiv \space Learning \space rate $

- $r_t  \equiv current \space reward \space at \space time \space step \space t .$

- $\gamma \equiv Discount \space factor.$ 

- $[ r_t +  \max_{a} \gamma Q_n(s_{t+1},a)] \equiv \space TD \space target $

- $ [r_{t} + \max_{a} \gamma Q_n(s_{t+1},a) - Q_{n-1}(s_t,a_t) ] \equiv  TD \space error.$


**Why the discount factor $\gamma$ ?**

That means, we do care about the current rewards more than long term rewards, the highest discount factor the more foucsing on the short rewards.

### **3.3 Q-learning algorithm:**

<p align="center">
  <img width="800" height="800" src="data/Q-learning.png">
</p>

### **3.4 Ways to implement Q-learning**

1. You can implement Q-learning using:

    - Q-Table ( Dictionary, Matrix ,.... ) : Each cell in the table represents action value in a given state.
             
|   |  |  Q-Table |    |
|---|---------------|------|-------|---
| $s_0$ |  $a_{00}$|  $a_{01}$|     $a_{02}$ | $a_{03}$ | ... |...$a_{0n}$...
| $s_1$ |    $a_{10}$|  $a_{11}$|     $a_{12}$ | $a_{13}$ | ... |...$a_{1n}$...
| $s_2$ |   $a_{20}$|  $a_{21}$|     $a_{22}$ | $a_{23}$ | ... |...$a_{2n}$...
| $s_3$|   $a_{30}$|  $a_{31}$|     $a_{32}$ | $a_{33}$ | ... |...$a_{3n}$...
| .. |  ..|  ..|      .. | .. | ... |...
        
   
   

   - Function Approximation: Because of the increasing of the size of Q-table with number of states, we approximate the action     value using **a function approximator like Neural Network**.

![Function approximator](data/func.jpg)

<hr>
# **4- Merge Q-learning with Deep Learning**

## **DQN:**

- Stands for **Deep Q-learning Network**. 
- A deep learning model to successfully learn control policies directly from high dimensional sensory input using Q-learning algorithm based method.
<hr>

### **4.1 Train the agent**

In [ ]:
# Create the Agent
agent = Agent(env=mountain_car)

In [ ]:
# Evaluation settings
episode_number = 0
max_episodes=1000
metric = Metric(max_episodes)

In [ ]:
while episode_number<max_episodes:
    
    metric.reset()
    
    R,episode_length= run(agent,mountain_car)
    
    metric.add(R,episode_number,episode_length)
 
    episode_number += 1
    
    
    if episode_number%100==0:
        
        metric.show()       

In [ ]:
# Save the agent
agent.brain.model.save('Models/dqn.mod')

In [ ]:
# Plot the result

plt.plot(metric.G)
plt.ylabel('Returns')
plt.xlabel('Number of episodes')

In [ ]:
plt.plot(metric.mean_G_all)
plt.ylabel('Average of Returns ')
plt.xlabel('Number of episodes')

In [ ]:
plt.plot(metric.episodes_length)
plt.ylabel('Episode Length')
plt.xlabel('Number of Episodes')

### **4.2 Test the agent**

Now, let's see how much our get better. **( uncomment and run this in your computer, not in colab)**

In [ ]:
# for i in range(20):
#     _,_= run(agent,mountain_car, train=False)

<hr>
# **5- Dive deeper into DQN: (Optional)**

In this section, we are going to implement DQN from scratch!. the section will introduce DQN components first, then we will try to stack these components together, to get a fully DQN algorithm.

## **5.1- DQN Components:**

**DQN algorithm consists of 3 main components:**

   1. Function approximator ( Neural Network ) .
   2. Reply buffer. ( Memory )
   3. Policy.

<p align="center">
  <img width="800" height="800" src="data/DQN.jpg">
</p>

### **1. Function approximator**

- Any machine learning algorithm can be seen as a function approximator, DQN algorithm uses the Neural Network as a function approximator. In the case of DQN the neural network outputs continuous value. 

- There are many deep learning frameoworks to implement the neural networks. This tutorial is based on Keras. Keras is a high API for other deep learning frameworks, such as Tensorflow, Theano and the CNTK.

- DQN uses the neural network to predict the action value in a given state $Q(s,a)$ . 
The Brain class represents the neural netowrks. It consists of 3 functions:
    -  create() : Which create the neural network.
    -  train():   train the model.
    -  predict(): predict the Q(s,a) value, it takes a state and output Q(s,a).

In [ ]:
class Brain:
    def __init__(self,state_size, action_size,batch_size,learning_rate):
        
        self.state_size = state_size
        self.action_size = action_size
        self.batch_size = batch_size
        self.lr = learning_rate
        
        self.params = {}
        self.model= self._create()
       
        
    def _create(self):
        
        model = Sequential()
        model.add(layers.Dense(units=64, activation='relu', input_dim= self.state_size))
        model.add(layers.Dense(units= self.action_size, activation='linear'))        
        
       
        optimizer = optimizers.adam(lr=self.lr)
        model.compile(loss='mse', optimizer=optimizer)
        

        return model

    def train(self, x, y, epoch=1, verbose=0):
        self.model.fit(x, y, batch_size=self.batch_size,epochs=epoch, verbose=verbose)
        

    def predict(self, s):
        
        return self.model.predict(s,batch_size=self.batch_size)


### **2. Experience Replay Buffer.**

- DQN use a memory to store the episode data **(transition tuple)** $(s_t , a_t , r_t, s_{t+1})$ as a training set. 
- Experience increase the stability of the learning.
- alleviates the problems of correlated data and non-stationary distributions.

In [ ]:
# Memory

class Memory:   # stored as ( s, a, r, s_ )
    samples = []

    def __init__(self, capacity):
        self.capacity = capacity

    def add_sample(self, sample):
        self.samples.append(sample)        

        if len(self.samples) > self.capacity:
            self.samples.pop(0)

    def get_sample(self, n):
        n = min(n, len(self.samples))
        return random.sample(self.samples, n)

### **3. Policy.**

- Policy: a policy is a function that map from state to action, not action value but action. $ \pi(s):a$
- In this tutorial, we implement the policy as a python class. The calss have 2 functions:
    - get_action: take an state and return the action.It return a random action with probability $\epsilon$ and an action based on the the output of the neural network with probability $1-\epsilon$.
    
    - decay_epsilon: which encourages the agent to take actions that not depending on its main policy. In the begining $\epsilon$ value is high which encourages the agent to take random actions, and through the time it gets to decay which let the agent take actions based on its own policy ( the Brain or Neural Network). 

In [ ]:
class Policy:

    def __init__(self, ACTION_COUNT):
        self.ACTION_COUNT = ACTION_COUNT
        self.max_epsilon = 1
        self.min_epsilon = 0.01
        self.epsilon = self.max_epsilon
        self.lmbda = 0.0001
        self.steps = 0
        pass
    
    def get_action(self,s,brain):
        
        if random.random() < self.epsilon:
            return random.randint(0, self.ACTION_COUNT-1)
        else:
            s=np.reshape(s,newshape=(1,s.shape[0]))
            return np.argmax(brain.predict(s)) 
    
    def decay_epsilon(self,steps):
        self.epsilon = self.max_epsilon + (self.max_epsilon - self.min_epsilon) * math.exp(-self.lmbda * self.steps)
        return self.epsilon

## **5.2  Stack the component together**

- In the previous section, we introduce DQN main components, in this section we will combine these components together to make DQN algorithm.

- To combine these components, we define a class called Agent, which will use all the previous code when it is intracing with environment. 

- Our Agent consists from 3 functions:
    - act(): take a state and return an action. it uses: get_action in policy class.
    - observe(): take  $(s_t , a_t , r_t, s_{t+1})$ and save in the memory or the Replay buffer. it uses:
        - add sample from Memory class.
        - decay_epsilon from Policy class.
        
    - replay(): update the neural network parameters. It uses:
        - get_sample in Memory class.
        - predict in Brain class.
        - train in Brain classs.
        
- Because Replay is an important function, this how it works:
    - It request data from the Memory, the number of samples from the memory called Batch. Each samples consist of $(s_t, a_t, r_t, s_{t+1})$.
   
    - Predict the actions values of all the $s_t$.
    - Predict the actions values of all the $s_{t+1}$.
    - Update the actions values of $s_t$.
    - Construct the data to update the Neural Network. data consists of:
        - X represents the states.
        - y represents the actions values that corsponding to each state. 
    - Update the Networks with the constructed data.
    

In [ ]:
class Agent:
    steps = 0
    MEMORY_CAPACITY = 100000
    BATCH_SIZE = 64
    GAMMA = 0.99
    lEARNING_RATE = 0.001
    
    def __init__(self, env):
        self.env = env
        self.STATE_COUNT  = self.env.observation_space.shape[0]
        self.ACTION_COUNT = self.env.action_space.n
        self.memory_capacity = 100000
        self.batch_size = 64
        self.lr = 0.001
        self.gamma = 0.99
    
        self.brain = Brain(self.STATE_COUNT,self.ACTION_COUNT,self.batch_size,self.lr)
        self.memory = Memory(self.memory_capacity)
        self.policy = Policy(self.ACTION_COUNT)
        
    def act(self, s):
        action=self.policy.get_action(s,self.brain)
        return action

    def observe(self, sample):  # in (s, a, r, s_) format
        self.memory.add_sample(sample)    
        # slowly decrease Epsilon based on our eperience
        self.steps += 1
        self.policy.decay_epsilon(self.steps)

    def replay(self):    
        batch = self.memory.get_sample(self.batch_size)
        batchLen = len(batch)

        no_state = np.zeros(self.STATE_COUNT)

        
        states = np.array([ o[0] for o in batch ], dtype=np.float32)
        states_ = np.array([(no_state if o[3] is None else o[3]) for o in batch ], dtype=np.float32)

        p = self.brain.predict(states)
        p_ = self.brain.predict(states_)

        x = np.zeros((batchLen, self.STATE_COUNT)).astype(np.float32)
        y = np.zeros((batchLen, self.ACTION_COUNT)).astype(np.float32)
        
        for i in range(batchLen):
            s, a, r, s_ = batch[i]
            
            action_values = p[i]
            if s_ is None:
                action_values[a] = r
                
            else:
                action_values[a] = r + self.gamma * np.amax(p_[i])      # calculate the target: r+ Gamma*max Q(s',a')

            x[i] = s
            y[i] = action_values

        self.brain.train(x, y)

## **5.3 Run the code**

- Follow these steps to implement the run function below:

    - Create an object from the Agent class.
    - Loop over the number of iterations.
    - Loop over the number of time step.
    - Use act() function to apply action in the environment through step() function.
    - Step() will returns the episode trajectory, save it in the memory.
    - Use replay() function to update the network.

In [ ]:
agent_sc = Agent(env=mountain_car)
max_episodes=1000
metric_sc = Metric(max_episodes)

In [ ]:
def run_agent(agent, env):
    s = env.reset()
    R = 0 
    episode_length_counter=1

    while True:     
        a = agent.act(s.astype(np.float32))

        s_, r, done, info = env.step(a)
        episode_length_counter+=1
        
        agent.observe((s, a, r, s_))
        agent.replay()            
        
        if done: # terminal state
            s_ = None
            env.close()
            return R,episode_length_counter
        
        s = s_
        R += r

In [ ]:
episode_number = 0

while episode_number<max_episodes:
    metric_sc.reset()
    
    R,episode_length = run_agent(agent_sc, mountain_car)
    
    metric_sc.add(R,episode_number,episode_length)

    episode_number += 1

    if episode_number%100==0:
        
        metric_sc.show()       

<hr>
# **6. The Big Picture of Reinforcement learning:**

## **6.1 General RL taxonomy**

- RL algorithms can be divided to 3 types:
    - Value functions based Methods.
    - Policy based Methods.
    - Hybrid Methods.

### **A) Value function Methods:**

- A value functions methods consider that every state or action has a value that can be learned, and then taking decisions based on those values. ( i.e taking actions based on the values of the actions) .

- As an example to value function methods: SARSA, Q-learning, DQN algorithms.

### **B) Policy function Methods:**

- Policy function methods don't need to learn values for actions or states to take decisions.
- The inputs are the state features and the ouputs are the actions or state . $\pi(s): a$
- They more stable more than Value function mthods, and can scale up in high dimensional state space and action space.
- An example to policy function methods: REINFORCE, DDPG algorithms.

### **C) Hybrid Methods:**

- These methods use both value function and policy function methods.
- It makes the learning even more stable than Policy iteration.
- Typically, two neural networks are used, one as a value function approximator and the other as policy function approximator.
- It needs more time time to be trained than the policy iteration.
- Example to some of these methods, ACTOR Critic algorithm.

## **6.2 Drawback of Value function Methods.**

- They can't be used in problems that have high dimension action space, because the number of the neural network outputs are increasing with the number of the actions

<hr>
# **7- References**

 - [Reinforcement Learning: An Introduction](http://incompleteideas.net/book/the-book-2nd.html) [Book]
 - [Deep Reinforcement Learning by Sergey Levine](http://rail.eecs.berkeley.edu/deeprlcourse/). [Course]
 - [Introductioon to Reinforcement learning by David Silver](http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching.html). [Course]
 - [Playing Atari with Deep Reinforcement Learning](https://arxiv.org/abs/1312.5602) [DQN paper]
 - [Simple Reinforcement Learning with Tensorflow](https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-0-q-learning-with-tables-and-neural-networks-d195264329d0) [blog]
 - [ An introduction to Reinforcement Learning](https://medium.freecodecamp.org/an-introduction-to-reinforcement-learning-4339519de419) [blog]

<hr>
# **8- Next Steps.**

- To see the current research problems in RL check this: [Deep RL does not work yet](https://www.alexirpan.com/2018/02/14/rl-hard.html). 